<a href="https://colab.research.google.com/github/CatarinaRRF/IC_design_de_siRNA/blob/main/code/algoritimo_siRNAseeker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://raw.githubusercontent.com/CatarinaRRF/IC_design_de_siRNA/main/media/Creative.png" height=100>
<hr color ='ACAD94' size="10">

## <font color=0A0908><b>Automatizando o design de RNAs</B>


<font color=6E7271><b>O que precisa ser feito</b></font>
<hr>

* Descobrir quais os parametros utilizados e discuitir a analise e uso do BLAST


## <font color=0A0908><b> 01. Introdução</b>

Com a crescente importância das pesquisas em biologia molecular e terapia genética, a identificação e seleção eficiente de pequenas moléculas de RNA interferente (siRNA) tornaram-se cruciais para a regulação precisa da expressão gênica. Neste contexto, é preciso criar um algoritmo que automatize e melhore a seleção de siRNA.

O algoritimo que será desenvolido utilizará do banco de dados mantido pelo NCBI <i>gene</i> onde, a partir dos arquivos inseridos pelo usuario, será analizado a viabilidade dos possiveis siRNA produzidos. Terá então 9 etapas principais, resumidas a seguir:

<br>

<img style="display: inline; margin: 15px 0;" title="heartica_logo" src= https://raw.githubusercontent.com/CatarinaRRF/IC_design_de_siRNA/main/media/fluxograma.png alt="" />

<b>Etapa 1: Entrada de Dados</b>

O algoritmo recebe um arquivo FASTA inserido pelo usuario contendo sequências de DNA que se deseja analisar. Cada sequência no arquivo FASTA é identificada por um cabeçalho (linha que começa com ">") e é seguida pela própria sequência.

<b>Etapa 2: Transcrição das Sequências de DNA</b>

Para a sequência de DNA encontrada no arquivo FASTA, o algoritmo realiza a transcrição, convertendo a sequência de DNA em uma sequência de RNA complementar.

<b>Etapa 3: Identificação de siRNA</b>

O algoritmo procura na sequência de RNA resultante por potenciais sequências de 22pb que serão potenciais siRNA.

<b>Etapa 4: Verificação da Qualidade do siRNA</b>

Cada sequência de siRNA identificada é submetida a uma análise de qualidade. Isso pode incluir a verificação da estrutura secundária, conteúdo de GC, estabilidade termodinâmica, e de posições de nucleotidios indicadas de acordo com a literatura consutada.

<b>Etapa 5: Excluir siRNA que não se enquadram nos critérios</b>

Os siRNAs que não atendem aos critérios de qualidade são excluídos da análise subsequente.

<b>Etapa 6: Execução do BLAST</b>

Para cada siRNA de qualidade, o algoritmo executa uma busca BLAST em um banco de dados de referência para encontrar alvos potenciais no transcriptoma. O BLAST ajuda a identificar possíveis correspondências com sequências conhecidas.

<b>Etapa 7: Extração de Informações do BLAST</b>

Após a execução do BLAST, o algoritmo extrai informações relevantes, como a sequência do alvo, o score do alinhamento e outros parâmetros importantes.

<b>Etapa 8: Criação de Tabela de Resultados</b>

Com as informações extraídas do BLAST, o algoritmo cria uma tabela de resultados que pode ser usada para análise posterior. Esta tabela inclui a sequência de siRNA, o score do BLAST, o algoritmo realiza a transcrição, convertendo a sequência de DNA em uma sequência de RNA complementar.

<b>Etapa 9: Fim</b>

O algoritmo conclui a análise das sequências de DNA, fornecendo uma tabela de resultados que pode ser usada para inferir a função ou alvo dos siRNAs identificados.



## <font color=0A0908><b> 02. Entrada de Dados</b>
<hr>


<font color=0A0908>O algoritmo recebe um arquivo FASTA inserido pelo usuario contendo sequências de DNA que se deseja analisar. Cada sequência no arquivo FASTA é identificada por um cabeçalho (linha que começa com ">") e é seguida pela própria sequência. Para esta sequencia de DNA, o algoritmo realiza a transcrição, convertendo a sequência de DNA em uma sequência de RNA complementar reverso.


<font color=ACAD94><b>Importando Bibliotecas

In [1]:
pip install biopython

In [2]:
import Bio
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import SeqIO

<font color=ACAD94><b> Entrada de Dados - Lendo o Arquivo Fasta

In [3]:
# selecionar o arquivo FASTA
nome_arquivo = "/content/sequence.fasta"

try:
    # Le o arquivo FASTA e armazena as sequencias em uma lista
    sequencias = list(SeqIO.parse(nome_arquivo, 'fasta'))

    # Verifica se o arquivo nao está vazio
    if len(sequencias) > 0:
        # Define o nome do arquivo de saída
        nome_arquivo_saida = 'meugene.fasta'

        # Salva as sequencias no arquivo de saida
        SeqIO.write(sequencias, nome_arquivo_saida, 'fasta')
        print(f'O arquivo {nome_arquivo} foi salvo como {nome_arquivo_saida}.')

    else:
        print('O arquivo FASTA está vazio.')
except Exception as e:
    print(f'Ocorreu um erro ao ler o arquivo FASTA: {str(e)}')

O arquivo /content/sequence.fasta foi salvo como meugene.fasta.

**************************** Informações ****************************
Nome: NM_004502.4
Descrição: NM_004502.4 Homo sapiens homeobox B7 (HOXB7), mRNA
Tamanho: 1363

**************************** Sequências ****************************
* Sequência de DNA:
 CTTTTTGGTGTAAATCTGGACTCTAATTCTGTAATATATCAAGGAATCTCGTAAAACCGACACTAAAACGTCCCTGCCTACAAATCATCCGGCCAAATTATGAGTTCATTGTATTATGCGAATACTTTATTTTCTAAATATCCAGCCTCAAGTTCGGTTTTCGCTACCGGAGCCTTCCCAGAACAAACTTCTTGTGCGTTTGCTTCCAACCCCCAGCGCCCGGGCTATGGAGCGGGTTCGGGCGCTTCCTTCGCCGCCTCGATGCAGGGCTTGTACCCCGGCGGGGGGGGCATGGCGGGCCAGAGCGCGGCCGGCGTCTACGCGGCCGGCTATGGGCTCGAGCCGAGTTCCTTCAACATGCACTGCGCGCCCTTTGAGCAGAACCTCTCCGGGGTGTGTCCCGGCGACTCCGCCAAGGCGGCGGGCGCCAAGGAGCAGAGGGACTCGGACTTGGCGGCCGAGAGTAACTTCCGGATCTACCCCTGGATGCGAAGCTCAGGAACTGACCGCAAACGAGGCCGCCAGACCTACACCCGCTACCAGACCCTGGAGCTGGAGAAAGAATTTCACTACAATCGCTACCTGACGCGGCGGCGGCGCATCGAGATCGCGCACACGCTCTGCCTCACGGAAAGACAGATCAAGATTTGGTTTCAGAACCGGCGCATGAAGTGGAAAAA

<font color=ACAD94><b> Trascrevendo em RNA complementar reverso

In [ ]:
for sequencia in sequencias:
            print()
            print("*"*28, 'Informações', "*"*28)
            print(f'Nome: {sequencia.name}')
            print(f'Descrição: {sequencia.description}')
            print(f'Tamanho: {len(sequencia.seq)}')

            # Converte as letras T para U
            complementar = sequencia.seq.reverse_complement()
            sequencia_convertida = Bio.Seq.transcribe(complementar)

            print()
            print("*"*28, 'Sequências', "*"*28)
            print(f'* Sequência de DNA:\n {sequencia.seq}')
            print(f'\n* RNA complementar reverso:\n {sequencia_convertida}')

## <font color=0A0908><b> 03.Construção dos possiveis siRNA</b>
<hr>

O algoritmo procura na sequência de RNA resultante por potenciais sequências de 22pb que serão potenciais siRNA. E as salva em uma lista separada.

<font color=ACAD94><b>Achando as possiveis sequencias de siRNA

In [4]:
possiveis_siRNA = []
for index, sequence in enumerate(sequencia_convertida):
   f = index+22
   possiveis_siRNA.append(str(sequencia_convertida[index:f]))
   siRNA = possiveis_siRNA[:-21]

siRNA[-10:]

['CAGAAUUAGAGUCCAGAUUUAC',
 'AGAAUUAGAGUCCAGAUUUACA',
 'GAAUUAGAGUCCAGAUUUACAC',
 'AAUUAGAGUCCAGAUUUACACC',
 'AUUAGAGUCCAGAUUUACACCA',
 'UUAGAGUCCAGAUUUACACCAA',
 'UAGAGUCCAGAUUUACACCAAA',
 'AGAGUCCAGAUUUACACCAAAA',
 'GAGUCCAGAUUUACACCAAAAA',
 'AGUCCAGAUUUACACCAAAAAG']

## <font color=0A0908><b> 04. Classificação dos siRNA quando sua funcionalidade</b>
<hr>

Cada sequência de siRNA identificada é submetida a uma análise de qualidade. Esta categorização é baseada nos trabalhos de Reynolds et al. (2004), Ui-Tei et al. (2004) e Amarzguioui et al, onde estabeleceram critérios para determinar a funcionalidade das sequências de siRNA, que incluem o conteúdo de CG%, temperatura de melting e baixa estabilidade interna.

Se uma sequência atender a esses critérios, ela é considerada funcional. Os critérios são classificados em ordem de importância da seguinte forma:

| Rank | Critério                                   | Pontuação |
| ---- | ------------------------------------------ | --------- |
| 1    | Ausência de repetições invertidas (hairpin) | -         |
| 2    | Estabilidade interna baixa                 | -2 a 10   |
| 3    | Baixo teor de GC                           | 0 ou 4    |
| 4    | Posições específicas                       | -4 a 6    |

Assim, os siRNAs candidatos são avaliados de acordo com esses critérios para determinar sua viabilidade. Para o primeiro critério, os siRNAs que não atendem são descartados. Para os demais critérios, é atribuída uma pontuação: 10 pontos para o critério de estabilidade baixa, 4 ponto para o critério de baixo teor GC e 1 ponto para o critério 4.

É importante destacar que, no caso das posições específicas, cada posição que corresponde a um nucleotídeo ideal recebe 1 ponto, enquanto a não conformidade com essa posição retorna 0 pontos. Nas posições onde o nucleotídeo precisa ser específico, é deduzido 1 ponto se não estiver em conformidade.

A pontuação então sera trasformada em pencentual, onde apenas as sequencias com mais de 80% de conformidade seram levadas para as proximas fases.

<hr>

> <font size="2.5"> codigo usado como base de biologyguy in <a href="https://github.com/biologyguy/public_scripts/blob/master/siRNA_predict.py" rel="">github</a>

<font color=ACAD94><b>Importando Bibliotecas

In [5]:
import re
from Bio.SeqUtils import MeltingTemp as mt
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq
import pandas as pd

<font color=ACAD94><b> Definindo uma função para a funcionalidade do siRNA

> Os paremetros que podem ser alterados são:
* Temperatura de Melting Maxima
* Intervalo do Conteudo CG
* A porcentagem em que é aceito os scores (80% conformidade, 85%, 90%, etc)

In [6]:
def siRNA_score (sequence, MaxTm, index):
  score = 0

  # Ausência de repetições invertidas Tm
  tm = round(mt.Tm_GC(sequence), 4)
  #-------------------------------------------------#

  # Estabilidade interna baixa
  for letra in sequence[14:19]:
    if letra == "A" or letra == "U":
      score += 2
    else:
      score -= 2
  #-------------------------------------------------#

  # Conteudo Baixo GC
  conteudo_gc = round(gc_fraction(sequence)*100, 2)
  if 30 <= conteudo_gc <= 52:
    score += 4
  #-------------------------------------------------#

  # Posições específicas - Critérios REYNOLDS et al. (2004) e AMARZGUIOUI et al. (2004)
  if sequence[0] == "A" or sequence[9] != "U":
    score += 1
  if sequence[5] == "A":
    score += 1
  else: score -= 1
  if sequence[9] == "U":
    score += 1
  if sequence[9] != "G":
    score += 1
  else: score -= 1
  if sequence[18] != "G" and sequence[18] != "C":
    score += 1
  else: score -= 1
  if sequence[18] == "A":
    score += 1
  else: score -= 1
  #-------------------------------------------------#

  return score, tm, conteudo_gc, MaxTm

<font color=ACAD94><b>Verificando cada sequencia de siRNA e salvando suas pontuações e parametros

In [7]:
# Definindo Variaveis
siRNA_verificados = []
score = []
tm = []
conteudo_gc = []
#-------------------------------------------------#

for index, sequence in enumerate(siRNA):
  #Veficando a qualidade
  resultado = siRNA_score(sequence, 60, index)

  #Exclui RNAs indesejadas
  if resultado[0] <= 16 or resultado[1] > resultado[3]:
    siRNA.pop(index)
  else:
  #Salvando os parametros
    siRNA_verificados.append(sequence)
    score.append(resultado[0])
    tm.append(resultado[1])
    conteudo_gc.append(resultado[2])

  # Printando as sequencias possiveis
    print(sequence, 'score:', resultado[0], 'Tm:', resultado[1])

UAUUUUUAAUUGCGUUUUAUUU score: 17 Tm: 38.2211
CCACAAAGACAGCAUUAAAGAG score: 19 Tm: 49.4029
ACAAAACUGCAAGAUUUUACAA score: 19 Tm: 43.812
ACAGAACAGGUAGAUAAUAUCC score: 17 Tm: 47.5393
AUAAUAUCCUUUUCAUAUUGUA score: 18 Tm: 40.0847
UUCAUAUUGUACAAAAAAACCA score: 19 Tm: 41.9484
UGUUGAAAAACCAAAAUUUCUC score: 17 Tm: 43.812
UCCUUAAAAAAUGUUUUUAAAC score: 19 Tm: 40.0847
CCUUAAAAAAUGUUUUUAAACU score: 19 Tm: 40.0847
CUUAAAAAAUGUUUUUAAACUC score: 19 Tm: 40.0847
UUUAAACUCCUUUCAUUUAAAU score: 19 Tm: 40.0847
UUAAACUCCUUUCAUUUAAAUA score: 17 Tm: 40.0847
UAAACUCCUUUCAUUUAAAUAG score: 17 Tm: 41.9484
CAUUUAAAUAGGGUUUUUUUUU score: 17 Tm: 40.0847
CUGGAUAUUUAGAAAAUAAAGU score: 17 Tm: 41.9484
AUAAAGUAUUCGCAUAAUACAA score: 18 Tm: 41.9484
AAUACAAUGAACUCAUAAUUUG score: 17 Tm: 41.9484
AUACAAUGAACUCAUAAUUUGG score: 17 Tm: 43.812
UACGAGAUUCCUUGAUAUAUUA score: 17 Tm: 43.812
ACGAGAUUCCUUGAUAUAUUAC score: 17 Tm: 45.6756
GAAUUAGAGUCCAGAUUUACAC score: 19 Tm: 47.5393


## <font color=0A0908><b>05. Blast</b>
<hr>


Para cada siRNA de qualidade, o algoritmo executa uma busca BLAST em um banco de dados de referência para encontrar alvos potenciais no transcriptoma. Após a execução, o algoritmo extrai informações relevantes, como a sequência do alvo, o score do alinhamento e outros parâmetros importantes.

> Aqui é importante ressaltar que os resultados serão impressos da mesma forma que seria o processo manual, em razão disso, seria interressante saber como são filtrados essas informações para que facilitação a automatização desse processo.

> Também gostariamos de saber sobre a questão de organismos se precisa ser especificado ou não.

<font color=ACAD94><b> Importando as Bibliotecas

In [8]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

In [10]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO

<font color=ACAD94><b> Criando um fasta contendo todas as sequências

In [9]:
records = []
for index, sequence in enumerate(siRNA_verificados):
    # Criando o cabeçalho enumerado
    header = f"sequencia_{index+1}"

    # Criando um objeto SeqRecord com a sequência e o cabeçalho
    record = SeqRecord(Seq(sequence), id=header, description="")

    # Adicionando o registro à lista
    records.append(record)

# Salvando os registros no arquivo FASTA
fasta_filename = "minha_sequencia.fasta"
with open(fasta_filename, "w") as output_file:
    SeqIO.write(records, output_file, "fasta")

print(f"As sequências foram salvas em '{fasta_filename}'.")


As sequências foram salvas em 'minha_sequencia.fasta'.


<font color=ACAD94><b>Rodando o BLAST

In [11]:
# Ler o arquivo fasta
fasta_file = "minha_sequencia.fasta"
sequences = list(SeqIO.parse(fasta_file, "fasta"))

# Configuração do BLAST
db = "refseq_rna"

# Realizar o BLAST para cada sequência
for i, sequence in enumerate(sequences, start=1):
    print(f"Processando sequência {i}...")
    result_handle = NCBIWWW.qblast("blastn", db, sequence.seq)

    # Parsear os resultados do BLAST
    from Bio.Blast import NCBIXML
    blast_records = NCBIXML.parse(result_handle)

    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            hit = alignment.hsps[0]  # Pega apenas o melhor alinhamento

            # Extrair informações relevantes
            description = alignment.title
            scientific_name = alignment.hit_def.split('[')[0]
            max_score = hit.score
            total_score = alignment.hsps[-1].score  # Pega o último HSP (Score total)
            query_cover = hit.align_length / len(sequence.seq)
            e_value = hit.expect
            per_ident = hit.identities / hit.align_length
            acc_len = alignment.length
            accession = alignment.accession

            # Imprimir os resultados
            print(f"Description: {description}")
            print(f"Scientific Name: {scientific_name}")
            print(f"Max Score: {max_score}")
            print(f"Total Score: {total_score}")
            print(f"Query Cover: {query_cover}")
            print(f"E value: {e_value}")
            print(f"Per.Ident: {per_ident}")
            print(f"Acc. Len: {acc_len}")
            print(f"Accession: {accession}")
            print("\n" + "=" * 50 + "\n")

    result_handle.close()

print("Processo concluído.")

A saída de streaming foi truncada nas últimas 5000 linhas.
Scientific Name: PREDICTED: Cyprinodon variegatus G kinase anchoring protein 1 (gkap1), transcript variant X2, mRNA
Max Score: 39.0
Total Score: 39.0
Query Cover: 1.0
E value: 9.30729
Per.Ident: 0.9545454545454546
Acc. Len: 1707
Accession: XM_015374263


Description: gi|974064798|ref|XM_015374262.1| PREDICTED: Cyprinodon variegatus G kinase anchoring protein 1 (gkap1), transcript variant X1, mRNA
Scientific Name: PREDICTED: Cyprinodon variegatus G kinase anchoring protein 1 (gkap1), transcript variant X1, mRNA
Max Score: 39.0
Total Score: 39.0
Query Cover: 1.0
E value: 9.30729
Per.Ident: 0.9545454545454546
Acc. Len: 1710
Accession: XM_015374262


Description: gi|755536489|ref|XM_011243737.1| PREDICTED: Mus musculus fidgetin-like 1 (Fignl1), transcript variant X2, mRNA
Scientific Name: PREDICTED: Mus musculus fidgetin-like 1 (Fignl1), transcript variant X2, mRNA
Max Score: 39.0
Total Score: 39.0
Query Cover: 1.0
E value: 9.30729

## <font color=0A0908><b> 06. Dataset e Visualização</b>
<hr>

Com as informações extraídas do BLAST, o algoritmo cria uma tabela de resultados que pode ser usada para análise posterior. Esta tabela inclui a sequência de siRNA, o score, a temperatura de melting e o conteudo CG. Observe que os parametros blast não estão adicionados, pelas razões previamente discutidas, porém, podem ser adicionados os seguintes criterios de acordo com a necessidade:

* Max score
* Cobertura da query
* E-value
* Identidade

<font color=ACAD94><b> Importando as Bibliotecas

In [12]:
import pandas as pd
#from google.colab import data_table

<font color=ACAD94><b> Criando um Dataset

In [15]:
dataset = pd.DataFrame({"Sequência 5'➔ 3'": siRNA_verificados,
                        "Pontuação": score,
                        "Temperatura de Melting": tm,
                        "Conteudo CG": conteudo_gc,
                        })
dataset = dataset.sort_values(by=["Pontuação"], ascending=False, ignore_index=True)

<font color=ACAD94><b> Prototipo do produto final

In [16]:
dataset

,Sequência 5'➔ 3',Pontuação,Temperatura de Melting,Conteudo CG
0,UUUAAACUCCUUUCAUUUAAAU,19,40.0847,36.36
1,UCCUUAAAAAAUGUUUUUAAAC,19,40.0847,30.77
2,CCACAAAGACAGCAUUAAAGAG,19,49.4029,45.00
3,CUUAAAAAAUGUUUUUAAACUC,19,40.0847,30.77
4,CCUUAAAAAAUGUUUUUAAACU,19,40.0847,30.77
5,GAAUUAGAGUCCAGAUUUACAC,19,47.5393,50.00
6,UUCAUAUUGUACAAAAAAACCA,19,41.9484,31.25
7,ACAAAACUGCAAGAUUUUACAA,19,43.8120,35.29
8,AUAAUAUCCUUUUCAUAUUGUA,18,40.0847,36.36
9,AUAAAGUAUUCGCAUAAUACAA,18,41.9484,31.25


<hr>
<font color=6E7271><b> ➔ Para mais informações visite o <a href="https://github.com/CatarinaRRF/IC_design_de_siRNA">github</a></b></font>
